In [4]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Read the CSV file into a DataFrame
data  = pd.read_csv('./dataset/datamusic1.csv')

In [5]:
# Display the first few rows of the dataset
data.head()

,Chroma,Spectral Centroid,Spectral Bandwidth,Spectral Rolloff,Zero-Crossing Rate,Tempo,Mel_Spectrogram_1,Mel_Spectrogram_2,Mel_Spectrogram_3,Mel_Spectrogram_4,Mel_Spectrogram_5,Mel_Spectrogram_6,Mel_Spectrogram_7,Mel_Spectrogram_8,Mel_Spectrogram_9,Mel_Spectrogram_10,Mel_Spectrogram_11,Mel_Spectrogram_12,Mel_Spectrogram_13,Mel_Spectrogram_14,Mel_Spectrogram_15,Mel_Spectrogram_16,Mel_Spectrogram_17,Mel_Spectrogram_18,Mel_Spectrogram_19,Mel_Spectrogram_20,Mel_Spectrogram_21,Mel_Spectrogram_22,Mel_Spectrogram_23,Mel_Spectrogram_24,Mel_Spectrogram_25,Mel_Spectrogram_26,Mel_Spectrogram_27,Mel_Spectrogram_28,Mel_Spectrogram_29,Mel_Spectrogram_30,Mel_Spectrogram_31,Mel_Spectrogram_32,Mel_Spectrogram_33,Mel_Spectrogram_34,Mel_Spectrogram_35,Mel_Spectrogram_36,Mel_Spectrogram_37,Mel_Spectrogram_38,Mel_Spectrogram_39,Mel_Spectrogram_40,Mel_Spectrogram_41,Mel_Spectrogram_42,Mel_Spectrogram_43,Mel_Spectrogram_44,Mel_Spectrogram_45,Mel_Spectrogram_46,Mel_Spectrogram_47,Mel_Spectrogram_48,Mel_Spectrogram_49,Mel_Spectrogram_50,Mel_Spectrogram_51,Mel_Spectrogram_52,Mel_Spectrogram_53,Mel_Spectrogram_54,Mel_Spectrogram_55,Mel_Spectrogram_56,Mel_Spectrogram_57,Mel_Spectrogram_58,Mel_Spectrogram_59,Mel_Spectrogram_60,Mel_Spectrogram_61,Mel_Spectrogram_62,Mel_Spectrogram_63,Mel_Spectrogram_64,Mel_Spectrogram_65,Mel_Spectrogram_66,Mel_Spectrogram_67,Mel_Spectrogram_68,Mel_Spectrogram_69,Mel_Spectrogram_70,Mel_Spectrogram_71,Mel_Spectrogram_72,Mel_Spectrogram_73,Mel_Spectrogram_74,Mel_Spectrogram_75,Mel_Spectrogram_76,Mel_Spectrogram_77,Mel_Spectrogram_78,Mel_Spectrogram_79,Mel_Spectrogram_80,Mel_Spectrogram_81,Mel_Spectrogram_82,Mel_Spectrogram_83,Mel_Spectrogram_84,Mel_Spectrogram_85,Mel_Spectrogram_86,Mel_Spectrogram_87,Mel_Spectrogram_88,Mel_Spectrogram_89,Mel_Spectrogram_90,Mel_Spectrogram_91,Mel_Spectrogram_92,Mel_Spectrogram_93,Mel_Spectrogram_94,Mel_Spectrogram_95,Mel_Spectrogram_96,Mel_Spectrogram_97,Mel_Spectrogram_98,Mel_Spectrogram_99,Mel_Spectrogram_100,Mel_Spectrogram_101,Mel_Spectrogram_102,Mel_Spectrogram_103,Mel_Spectrogram_104,Mel_Spectrogram_105,Mel_Spectrogram_106,Mel_Spectrogram_107,Mel_Spectrogram_108,Mel_Spectrogram_109,Mel_Spectrogram_110,Mel_Spectrogram_111,Mel_Spectrogram_112,Mel_Spectrogram_113,Mel_Spectrogram_114,Mel_Spectrogram_115,Mel_Spectrogram_116,Mel_Spectrogram_117,Mel_Spectrogram_118,Mel_Spectrogram_119,Mel_Spectrogram_120,Mel_Spectrogram_121,Mel_Spectrogram_122,Mel_Spectrogram_123,Mel_Spectrogram_124,Mel_Spectrogram_125,Mel_Spectrogram_126,Mel_Spectrogram_127,Mel_Spectrogram_128,MFCC_1,MFCC_2,MFCC_3,MFCC_4,MFCC_5,MFCC_6,MFCC_7,MFCC_8,MFCC_9,MFCC_10,MFCC_11,MFCC_12,MFCC_13,Label
0,0.272263,1784.122641,2002.412407,3805.723030,0.083045,123.046875,6.014573,31.485012,19.837402,14.071877,35.668877,18.231876,27.470974,25.481300,14.729004,13.210428,13.599668,6.014392,6.108338,10.312270,22.726816,14.130291,33.005290,45.761074,10.804842,10.045342,9.610369,16.888021,15.029155,7.078440,13.669671,15.452975,13.124710,6.209235,5.148327,6.184896,3.296579,2.268045,3.523205,3.706431,2.750797,3.318183,2.539076,4.016566,2.084159,1.587797,2.150195,2.759289,2.974856,5.556302,4.030466,1.623634,0.867555,0.772729,1.379876,0.941097,2.339987,2.187452,1.377493,0.531487,0.690711,0.973008,0.529387,0.253276,0.669768,1.914238,0.561936,1.122298,1.576224,2.027921,0.798679,0.920159,0.587863,0.208748,0.251220,0.975096,1.292337,0.504068,0.262326,0.313301,0.407954,0.440301,0.426866,0.267106,0.244592,0.135104,0.184391,0.174616,0.266293,0.224820,0.191117,0.156204,0.073564,0.065935,0.060123,0.053808,0.055240,0.059651,0.079378,0.072307,0.048550,0.048054,0.048486,0.037782,0.035334,0.035362,0.028285,0.026161,0.022250,0.032642,0.025666,0.024625,0.021038,0.022766,0.027336,0.019131,0.030917,0.043772,0.033502,0.027264,0.020595,0.009887,0.006498,0.005379,0.005652,0.010523,0.009256,0.001532,0.000426,0.000134,0.000019,1.853681e-06,1.897305e-07,4.178686e-08,-113.598820,121.57067,-19.162262,42.363940,-6.362266,18.621931,-13.699734,15.339802,-12.274305,10.970944,-8

In [6]:
# Check the columns in the dataset
data.columns


Index(['Chroma', 'Spectral Centroid', 'Spectral Bandwidth', 'Spectral Rolloff',
       'Zero-Crossing Rate', 'Tempo', 'Mel_Spectrogram_1', 'Mel_Spectrogram_2',
       'Mel_Spectrogram_3', 'Mel_Spectrogram_4',
       ...
       'MFCC_5', 'MFCC_6', 'MFCC_7', 'MFCC_8', 'MFCC_9', 'MFCC_10', 'MFCC_11',
       'MFCC_12', 'MFCC_13', 'Label'],
      dtype='object', length=148)

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pickle
import librosa
import numpy as np

# Function to extract features
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None)
    features = {
        'Chroma': np.mean(librosa.feature.chroma_stft(y=y, sr=sr)),
        'RMSE': np.mean(librosa.feature.rms(y=y)),
        'Spectral_Centroid': np.mean(librosa.feature.spectral_centroid(y=y, sr=sr)),
        'Spectral_Bandwidth': np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr)),
        'RollOff': np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr)),
        'Zero_Crossing_Rate': np.mean(librosa.feature.zero_crossing_rate(y)),
        'Harmony': np.mean(librosa.effects.harmonic(y)),
        'Percussive': np.mean(librosa.effects.percussive(y)),
        'Tempo': librosa.beat.tempo(y=y, sr=sr)[0]
    }
    for i, mfcc in enumerate(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)):
        features[f'MFCC_{i+1}'] = np.mean(mfcc)
    return features

# Load dataset
file_path = './dataset/datamusic1.csv'
data = pd.read_csv(file_path)

# Check if 'file_path' column exists
if 'file_path' not in data.columns:
    raise KeyError("The dataset does not contain a 'file_path' column. Please ensure the dataset is structured correctly.")

# Extract features for the entire dataset
data_features = []
for index, row in data.iterrows():
    file_path = row['file_path']  # Assuming 'file_path' column exists
    features = extract_features(file_path)
    features['Label'] = row['Label']
    data_features.append(features)

# Create DataFrame from features
features_df = pd.DataFrame(data_features)

# Separate features and label
X = features_df.drop(columns=['Label'])
y = features_df['Label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Save the trained model
model_path = '/mnt/data/random_forest_model.pkl'
with open(model_path, 'wb') as model_file:
    pickle.dump(clf, model_file)

# Optionally, you can test the model accuracy on the test set
accuracy = clf.score(X_test, y_test)
print(f"Model accuracy: {accuracy}")


KeyError: "The dataset does not contain a 'file_path' column. Please ensure the dataset is structured correctly."

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pickle

# Load dataset
file_path = './dataset/datamusic1.csv'
data = pd.read_csv(file_path)

# Separate features (X) and labels (y)
X = data.drop(columns=['Label'])
y = data['Label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Save the trained model
model_path = '/mnt/data/random_forest_model.pkl'
with open('random_forest_model.pkl', 'wb') as model_file:
    pickle.dump(clf, model_file)

# Optionally, you can test the model accuracy on the test set
accuracy = clf.score(X_test, y_test)
print(f"Model accuracy: {accuracy}")
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pickle

# Load dataset
file_path = './dataset/datamusic1.csv'
data = pd.read_csv(file_path)

# Separate features (X) and labels (y)
X = data.drop(columns=['Label'])
y = data['Label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Save the trained model
model_path = 'random_forest_model.pkl'
with open('random_forest_model.pkl', 'wb') as model_file:
    pickle.dump(clf, model_file)

# Optionally, you can test the model accuracy on the test set
accuracy = clf.score(X_test, y_test)
print(f"Model accuracy: {accuracy}")


Model accuracy: 0.61
Model accuracy: 0.61
